In [ ]:
import math
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup 

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
cols = ['title', 'company', 'location', 'salary', 'days_since_posted', 'link', 'career_level', 'qualification',
        'years_of_experience', 'job_type', 'specialization', 'subspecialty', 'night_shift', 'sql', 'excel', 'tableau', 
        'power_bi', 'python', 'r']
title = []
company = []
location = []
salary = []
days_since_posted = []
link = []
career_level = []
qualification = []
years_of_experience = []
job_type = []
specialization = []
subspecialty = []
night_shift = []
sql = []
excel = []
tableau = []
power_bi = []
python = []
r = []
extracted_info = []
    
def jobs_extract(job_list):    
    for job_post in job_list:        
        job_info = ['none', 'none', 'none', 'none', 'none', 'none', 'none', 'none', 'none', 'none', 'none', 'none',
                    'none', 'none', 'none', 'none', 'none', 'none', 'none']
        
        # Job Title
        job_info[0] = job_post.find('div', attrs={'class': 'z1s6m00 l3gun70 l3gun74 l3gun72'}).text.strip()
        print("\n" + job_info[0])
        
        # Company Name
        # sometimes data-automation attribute is not working
#         company.append(job_post.find('a', attrs={'data-automation': 'jobCardCompanyLink'}).text.strip()) 
        job_info[1] = job_post.find('span', attrs={'class': 'z1s6m00 _17dyj7u1 _1hbhsw64u _1hbhsw60 _1hbhsw6r'}).text.strip()
        
        # Location and Salary
#         location = job_post.find('a', attrs={'data-automation': 'jobCardLocationLink'}).text
        loc_salary_info = [
            location_salary.getText(strip=True) for location_salary in job_post.findAll('span', attrs={'class': 'z1s6m00 _1hbhsw64u y44q7i0 y44q7i3 y44q7i21 y44q7ih'})
        ]
        job_info[2] = loc_salary_info[0]
        print(job_info[2])
        
        if len(loc_salary_info) == 2:
            job_info[3] = loc_salary_info[1]
            print(job_info[3])
        else:
            print("none")
        
        # Days since posted
        job_info[4] = job_post.find('time', attrs={'class': 'z1s6m00 _1hbhsw64u'}).text.strip()
        
        # Full details
        details = job_post.find('a', attrs={'class': 'jdlu994 jdlu996 jdlu999 y44q7i2 z1s6m00 z1s6m0f _1hbhsw6h'}, href=True)

        job_post_url = 'https://www.jobstreet.com.ph' + str(details['href'])
        job_post_page = requests.get(job_post_url, headers=headers)
        job_info[5] = job_post_url
        
        job_post_soup1 = BeautifulSoup(job_post_page.content,'html.parser')
        job_post_soup2 = BeautifulSoup(job_post_soup1.prettify(), 'html.parser')

        job_post_div = job_post_soup2.findAll('div', attrs={'data-automation': 'jobDescription'})
        
        for job_desc in job_post_div:
            description = job_desc.text

            if 'night shift' in description.lower():
                job_info[12] = 'Yes'
            else:
                job_info[12] = 'No'
                
            if 'sql' in description.lower():
                job_info[13] = 'Yes'
            else:
                job_info[13] = 'No'
                
            if 'excel' in description.lower():
                job_info[14] = 'Yes'
            else:
                job_info[14] = 'No'
                
            if 'tableau' in description.lower():
                job_info[15] = 'Yes'
            else:
                job_info[15] = 'No'
                
            if 'power bi' in description.lower():
                job_info[16] = 'Yes'
            else:
                job_info[16] = 'No'

            if 'python' in description.lower():
                job_info[17] = 'Yes'
            else:
                job_info[17] = 'No'

            if re.search("[^a-zA-Z]R[^a-zA-Z]", description):
                job_info[18] = 'Yes'
            else:
                job_info[18] = 'No'
                                
        # Additional job information
        job_post_div2 = job_post_soup2.findAll('div', attrs={'class': 'z1s6m00 _1hbhsw6r pmwfa50 pmwfa57'})
        
        for job_addtl_desc in job_post_div2:
            skills2 = job_addtl_desc.text

            if 'career level' in skills2.lower():
                pattern = re.compile('Career Level', re.IGNORECASE)
                job_info[6] = pattern.sub('', skills2).strip()             
            elif 'qualification' in skills2.lower():
                pattern = re.compile('Qualification', re.IGNORECASE)
                job_info[7] = pattern.sub('', skills2).strip()             
            elif 'years of experience' in skills2.lower():
                pattern = re.compile('Years of Experience', re.IGNORECASE)
                job_info[8] = pattern.sub('', skills2).strip()             
            elif 'job type' in skills2.lower():
                pattern = re.compile('Job Type', re.IGNORECASE)
                job_info[9] = pattern.sub('', skills2).strip()             
            elif 'job specializations' in skills2.lower():
                pattern = re.compile('Job Specializations', re.IGNORECASE)
                specializations = pattern.sub('', skills2).split(',')
                job_info[10] = specializations[0].strip()
                job_info[11] = specializations[1].strip()
                
        #Save all additional information
        title.append(job_info[0])
        company.append(job_info[1])
        location.append(job_info[2])
        salary.append(job_info[3])
        days_since_posted.append(job_info[4])
        link.append(job_info[5])
        career_level.append(job_info[6])
        qualification.append(job_info[7])
        years_of_experience.append(job_info[8])
        job_type.append(job_info[9])
        specialization.append(job_info[10])
        subspecialty.append(job_info[11])
        night_shift.append(job_info[12])
        sql.append(job_info[13])
        excel.append(job_info[14])
        tableau.append(job_info[15])
        power_bi.append(job_info[16])
        python.append(job_info[17])
        r.append(job_info[18])
        
        print("python? " + job_info[17])
        print("r? " + job_info[17])
    
def jobs_save():
    #Save all information
    all_jobs_list = {}

    print("Length of columns")
    print(len(title))
    print(len(company))
    print(len(location))
    print(len(salary))
    print(len(days_since_posted))
    print(len(link))
    print(len(career_level))
    print(len(qualification))
    print(len(years_of_experience))
    print(len(job_type))
    print(len(specialization))
    print(len(subspecialty))
    print(len(night_shift))
    print(len(sql))
    print(len(excel))
    print(len(tableau))
    print(len(power_bi))
    print(len(python))
    print(len(r))
      
    extracted_info.append(title)
    extracted_info.append(company)
    extracted_info.append(location)
    extracted_info.append(salary)
    extracted_info.append(days_since_posted)
    extracted_info.append(link)
    extracted_info.append(career_level)
    extracted_info.append(qualification)
    extracted_info.append(years_of_experience)
    extracted_info.append(job_type)
    extracted_info.append(specialization)
    extracted_info.append(subspecialty)
    extracted_info.append(night_shift)
    extracted_info.append(sql)
    extracted_info.append(excel)
    extracted_info.append(tableau)
    extracted_info.append(power_bi)
    extracted_info.append(python)    
    extracted_info.append(r)    

    for j in range(len(cols)):
        all_jobs_list[cols[j]] = extracted_info[j]

    filename = "jobstreet_dataset.xls"
    jobs = pd.DataFrame(all_jobs_list)
    jobs.to_excel(filename, index=False)    
                                            
def jobs_search():
    page_number = 1
    total_num_pages = 1 #Initial number of pages to be extracted
    jobs_per_page = 30 #Number of jobs per page

    while page_number <= total_num_pages:
        url = 'https://www.jobstreet.com.ph/en/job-search/data-analyst-jobs-in-philippines/' + str(page_number)
        page = requests.get(url, headers=headers)
        soup1 = BeautifulSoup(page.content,"html.parser")
        soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

        job_list = soup2.findAll('div', attrs={'class': 'z1s6m00 _1hbhsw69e _1hbhsw68e _1hbhsw672 _1hbhsw67e'})
#         job_list = soup2.findAll('div', attrs={'data-automation': 'jobListing'})

        # No. of jobs
        if 1 == page_number:
            job_search_bar = soup2.find('div', attrs={'data-automation': 'searchResultBar'}).text.split('of')
            job_numbers = int(job_search_bar[1].replace('jobs', '').replace(',', '').strip())
            total_num_pages = math.ceil(job_numbers/jobs_per_page)
            print(total_num_pages)
        
        print("Page number: " + str(page_number))
        page_number += 1
            
        jobs_extract(job_list)
    
    jobs_save()
                
jobs_search()

In [ ]:
%%javascript

window.scroll_flag = true
window.scroll_exit = false
window.scroll_delay = 100

$(".output_scroll").each(function() {
    $(this)[0].scrollTop = $(this)[0].scrollHeight;
});

function callScrollToBottom() {
    setTimeout(scrollToBottom, window.scroll_delay);
}

function scrollToBottom() {
    if (window.scroll_exit) {
        return;
    }
    if (!window.scroll_flag) {
        callScrollToBottom();
        return;
    };
    
    $(".output_scroll").each(function() {
        if (!$(this).attr('scroll_checkbox')){
            window.scroll_flag = true;
            $(this).attr('scroll_checkbox',true);
            var div = document.createElement('div');
            var checkbox = document.createElement('input');
            checkbox.type = "checkbox";
            checkbox.onclick = function(){window.scroll_flag = checkbox.checked}
            checkbox.checked = "checked"
            div.append("Auto-Scroll-To-Bottom: ");
            div.append(checkbox);
            $(this).parent().before(div);
        }
        
        $(this)[0].scrollTop = $(this)[0].scrollHeight;
    });
    callScrollToBottom();
}
scrollToBottom();